<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/StepFunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np

### Environment

In [8]:
class MyEnv(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=Box(low=np.array([0]),high=np.array([50]),dtype=np.float32) 
        self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_capacity=50
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(120,150)#df['demand']
        power=np.random.randint(60,110)#df['Solar_gen']
        net_load=max((load-power),0)
        net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        battery_chng=0
        if action== 0:
          grid_export_=self.battery_cap+power-load-self.max_battery_capacity # grid export
          battery_chng=abs(self.battery_efficiency*self.max_battery_capacity)#change in battery
          self.battery_cap=min(max(self.battery_efficiency*self.max_battery_capacity,0),50) # battery charge
          info={
              'grid_export':grid_export_,
              'battery_change':battery_chng,
              'grid_import':0
          }
        if action== 1:
          battery_chng=abs(self.battery_efficiency*(power-load))#change in battery
          self.battery_cap=min(max(self.battery_cap + self.battery_efficiency*(power-load),0),50)# only battery charge
          info={
              'grid_export':0,
              'battery_change':battery_chng,
              'grid_import':0
          }
        if action== 2:
          grid_import_=net_rem #grid import
          battery_chng=abs(self.battery_efficiency*self.battery_cap)#change in battery
          self.battery_cap=min(max(self.battery_cap-self.battery_efficiency*self.battery_cap,0),50)# battery discharge
          info={
              'grid_export':0,
              'battery_change':battery_chng,
              'grid_import':grid_import_
             }
        if action== 3:
          battery_chng=abs(self.battery_efficiency*(load-power))#change in battery
          self.battery_cap=min(max(self.battery_cap-(load-power),0),50) #only battery discharge
          info={
              'grid_export':0,
              'battery_change':battery_chng,
              'grid_import':0
                    }     
        
        reward=grid_export_*8-grid_import_*10-(battery_chng*0.5)
        
        if(self.battery_cap<2.0):
              done=True
      
        return self.battery_cap,reward, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap


In [9]:
env=MyEnv()

### Agent

In [11]:
for e in range(50): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        #print(action)
        observation,reward,done,info = env._step(action)
        #print(observation)
        print(info)
        score+=round(reward)
        
        #print(info['load'],info['power'])
      
    print('episode:{} score:{}'.format(e,score))


{'grid_export': -57, 'battery_change': 15.0, 'grid_import': 0}
{'grid_export': 0, 'battery_change': 14.399999999999999, 'grid_import': 0}
episode:0 score:-471
{'grid_export': 0, 'battery_change': 9.0, 'grid_import': 61.0}
{'grid_export': -42.0, 'battery_change': 15.0, 'grid_import': 0}
{'grid_export': 0, 'battery_change': 18.3, 'grid_import': 0}
episode:1 score:-967
{'grid_export': 0, 'battery_change': 9.0, 'grid_import': 72.0}
{'grid_export': 0, 'battery_change': 6.3, 'grid_import': 56.7}
{'grid_export': 0, 'battery_change': 4.409999999999999, 'grid_import': 68.59}
{'grid_export': -55.71000000000001, 'battery_change': 15.0, 'grid_import': 0}
{'grid_export': 0, 'battery_change': 4.5, 'grid_import': 63.5}
{'grid_export': 0, 'battery_change': 23.099999999999998, 'grid_import': 0}
episode:2 score:-3084
{'grid_export': 0, 'battery_change': 9.0, 'grid_import': 53.0}
{'grid_export': 0, 'battery_change': 6.3, 'grid_import': 37.7}
{'grid_export': 0, 'battery_change': 4.409999999999999, 'grid_i